# Stacks

**The core idea:** A stack is a collection where the last item in is the first item out. **LIFO — Last In, First Out**. Think of a stack of plates — you add to the top, you take from the top. You never touch the bottom.


In [1]:
def println()->None:
    print('-'*50)

In [3]:
stack = []
stack.append("a")    # push — add to top
print(stack)
stack.append("b")
print(stack)
stack.append("c")
print(stack)
stack.pop()          # pop — remove from top → "c"
print(stack)
stack.pop()          # → "b"
print(stack)

['a']
['a', 'b']
['a', 'b', 'c']
['a', 'b']
['a']


That's it. A stack in Python is just a list using only `append()` and `pop()`. No new syntax. The constraint is the concept.

**The three operations:**

In [5]:
stack = []

# Push
stack.append(1)
stack.append(2)
stack.append(3)      # stack is [1, 2, 3]

# Pop
stack.pop()          # returns 3, stack is [1, 2]

# Peek — look at top without removing
stack[-1]            # 2 — top of stack

# Check empty
len(stack) == 0      # or: not stack
print(not stack)

False




**Why stacks exist — the mental model:**

Any problem where you need to **remember where you came from** or **undo the last thing** is a stack problem. The most classic example — bracket matching:


Walk through `"([)}"`:
- `(` → push → stack: `["("]`
- `[` → push → stack: `["(", "["]`
- `)` → top is `[`, expected `(` → mismatch → return False

This is LeetCode #20 — Valid Parentheses. One of the most common easy problems. Pure stack.


**Real world uses you already know:**

Your browser back button is a stack. Ctrl+Z undo is a stack. The call stack when Python runs your functions is a stack — that's why you get "stack overflow" when recursion goes too deep.




In [10]:
def is_valid_brackets(s):
    stack = []
    pairs = {")": "(", "]": "[", "}": "{"}
    
    for char in s:
        if char in pairs.values():
            stack.append(char)        # push opening bracket
        elif char in pairs.keys():
            if not stack or stack[-1] != pairs[char]:
                return False
            stack.pop()               # pop matching opening bracket
    
    return len(stack) == 0            # valid if nothing left
# === TEST CASES ===

# ✅ Valid cases
assert is_valid_brackets("()") == True
assert is_valid_brackets("()[]{}") == True
assert is_valid_brackets("{[]}") == True
assert is_valid_brackets("({[]})") == True
assert is_valid_brackets("((()))") == True
assert is_valid_brackets("[{()}]") == True
assert is_valid_brackets("") == True                    # empty string
assert is_valid_brackets("abc") == True                 # no brackets at all

# ❌ Invalid cases
assert is_valid_brackets("(]") == False                 # mismatched pair
assert is_valid_brackets("([)]") == False               # wrong nesting order
assert is_valid_brackets("(") == False                  # unclosed opening
assert is_valid_brackets(")") == False                  # unmatched closing
assert is_valid_brackets("((())") == False              # missing one closing
assert is_valid_brackets("}}") == False                 # double closing, no opening
assert is_valid_brackets("[") == False                  # single opening
assert is_valid_brackets("]") == False                  # single closing

print("All tests passed! ✅")

All tests passed! ✅


**`collections.deque` as a stack** — faster for large data:
For small problems a list is fine. For performance-critical code use `deque`.

In [12]:
from collections import deque
stack = deque()
stack.append(1)      # push
stack.pop()          # pop from right — same as list

1

**LeetCode patterns: 739 **

**Monotonic stack** — maintains order while processing:

![739_1](../images/739_1.png)

Monotonic means **consistently moving in one direction** — either always increasing or always decreasing. Never changing direction.

Think of it like a one-way ramp:

```
Monotonic increasing:  [1, 3, 5, 8, 12]  ✅  always going up
Monotonic decreasing:  [9, 7, 4, 2, 1]   ✅  always going down
Not monotonic:         [1, 3, 2, 5, 4]   ❌  goes up then down
```

In a **monotonic stack** you enforce this rule as you push elements — if a new element would break the order, you **pop** elements off until the order is restored, then push.

```python
# Monotonic increasing stack
stack = []
nums = [3, 1, 4, 1, 5]

for n in nums:
    while stack and stack[-1] > n:  # pop anything larger
        stack.pop()
    stack.append(n)

# Result: [1, 1, 5] — maintains increasing order
```

The popping step is the key — you're not just blindly pushing, you're **maintaining the invariant** that the stack stays ordered.

This pattern is powerful for problems like "find the next greater element" or "find how far you can see" — the monotonic property lets you answer those questions in O(n) instead of O(n²) nested loops. It's a classic interview pattern you'll see repeatedly in LeetCode.

In [47]:
def daily_temperatures(temps):
    stack = []      # stores indices
    result = [0] * len(temps)
    
    for i, temp in enumerate(temps):
        while stack and temps[stack[-1]] < temp:
            idx = stack.pop()
            result[idx] = i - idx
        stack.append(i)
    
    return result
print(daily_temperatures([73, 74, 75, 71, 69, 72, 76, 73]))
# Test 1: Basic case from LeetCode
temps1 = [73, 74, 75, 71, 69, 72, 76, 73]
expected1 = [1, 1, 4, 2, 1, 1, 0, 0]
assert daily_temperatures(temps1) == expected1, f"Test 1 failed: got {daily_temperatures(temps1)}"

# Test 2: Strictly increasing temperatures
temps2 = [30, 40, 50, 60]
expected2 = [1, 1, 1, 0]  # each day waits just 1 day, last day never warms
assert daily_temperatures(temps2) == expected2, f"Test 2 failed"

# Test 3: Strictly decreasing temperatures
temps3 = [60, 50, 40, 30]
expected3 = [0, 0, 0, 0]  # no warmer day ever comes
assert daily_temperatures(temps3) == expected3, f"Test 3 failed"

# Test 4: All same temperatures
temps4 = [50, 50, 50, 50]
expected4 = [0, 0, 0, 0]  # never gets warmer (must be STRICTLY warmer)
assert daily_temperatures(temps4) == expected4, f"Test 4 failed"

# Test 5: Single element
temps5 = [75]
expected5 = [0]  # no future days
assert daily_temperatures(temps5) == expected5, f"Test 5 failed"

# Test 6: Two elements, increasing
temps6 = [55, 60]
expected6 = [1, 0]
assert daily_temperatures(temps6) == expected6, f"Test 6 failed"

# Test 7: Two elements, decreasing
temps7 = [60, 55]
expected7 = [0, 0]
assert daily_temperatures(temps7) == expected7, f"Test 7 failed"

# Test 8: Empty list
temps8 = []
expected8 = []
assert daily_temperatures(temps8) == expected8, f"Test 8 failed"

# Test 9: Pattern with jumps
temps9 = [80, 60, 70, 90]
expected9 = [3, 1, 1, 0]
# Day 0 (80): waits 3 days for 90
# Day 1 (60): waits 1 day for 70
# Day 2 (70): waits 1 day for 90
# Day 3 (90): never warms
assert daily_temperatures(temps9) == expected9, f"Test 9 failed"

print("All tests passed! ✅")


[0, 1, 1, 4, 2, 1, 1, 0, 0]
All tests passed! ✅


Your logic is **correct** and shows you understood the problem well. But there are some performance and structural concerns worth discussing.

**What you got right:**
- `popleft()` correctly processes each day in order
- The inner loop finds the next warmer day
- `i+1` correctly calculates days waited
- Default 0 already in result if no warmer day found

**The problems:**

**1. Destroying your data** — `temps.popleft()` is consuming the deque as you go. So when you're checking day 2's future temperatures, day 0 and day 1 are already gone. This actually works here by accident because you only look **forward**, but it's fragile and confusing.

**2. O(n²) performance** — for every element you're scanning the remaining list. The monotonic stack solution does this in O(n). With a million temperatures yours crawls, the stack solution flies.

**3. `len(temps)` after `deque(temps)`** — once you start popping, `len(temps)` shrinks. You captured it correctly before the loop but it's a hidden trap.

**4. `j` is just reimplementing `enumerate`** — you're manually tracking what Python gives you for free.

Your solution is the **intuitive brute force approach** — which is always the right starting point. You understood the problem correctly. The monotonic stack is just the optimized version of exactly what you built.

In [48]:
#Sean's Implementation
from collections import deque
def daily_temperatures(temps):
    temps = deque(temps)
    #temps= temps[::-1]
    result = [0] * len(temps)
    j = 0
    while temps:
        cur = temps.popleft()
        numdays = 0
        for i, temp in enumerate(temps):
            if temp > cur:
                result[j] = i+1
                break 
        j += 1
    return result
            
            
  
# Test 1: Basic case from LeetCode
temps1 = [73, 74, 75, 71, 69, 72, 76, 73]
expected1 = [1, 1, 4, 2, 1, 1, 0, 0]
assert daily_temperatures(temps1) == expected1, f"Test 1 failed: got {daily_temperatures(temps1)}"

# Test 2: Strictly increasing temperatures
temps2 = [30, 40, 50, 60]
expected2 = [1, 1, 1, 0]  # each day waits just 1 day, last day never warms
assert daily_temperatures(temps2) == expected2, f"Test 2 failed"

# Test 3: Strictly decreasing temperatures
temps3 = [60, 50, 40, 30]
expected3 = [0, 0, 0, 0]  # no warmer day ever comes
assert daily_temperatures(temps3) == expected3, f"Test 3 failed"

# Test 4: All same temperatures
temps4 = [50, 50, 50, 50]
expected4 = [0, 0, 0, 0]  # never gets warmer (must be STRICTLY warmer)
assert daily_temperatures(temps4) == expected4, f"Test 4 failed"

# Test 5: Single element
temps5 = [75]
expected5 = [0]  # no future days
assert daily_temperatures(temps5) == expected5, f"Test 5 failed"

# Test 6: Two elements, increasing
temps6 = [55, 60]
expected6 = [1, 0]
assert daily_temperatures(temps6) == expected6, f"Test 6 failed"

# Test 7: Two elements, decreasing
temps7 = [60, 55]
expected7 = [0, 0]
assert daily_temperatures(temps7) == expected7, f"Test 7 failed"

# Test 8: Empty list
temps8 = []
expected8 = []
assert daily_temperatures(temps8) == expected8, f"Test 8 failed"

# Test 9: Pattern with jumps
temps9 = [80, 60, 70, 90]
expected9 = [3, 1, 1, 0]
# Day 0 (80): waits 3 days for 90
# Day 1 (60): waits 1 day for 70
# Day 2 (70): waits 1 day for 90
# Day 3 (90): never warms
assert daily_temperatures(temps9) == expected9, f"Test 9 failed"

print("All tests passed! ✅")

All tests passed! ✅
